In [26]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import cifar10
from tqdm import tqdm

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [27]:
print(x_train.shape)

(50000, 32, 32, 3)


In [28]:
# Convert images to grayscale
x_train_gray = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in x_train])
x_test_gray = np.array([cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in x_test])

# Reshape images into 1D arrays
x_train_flat = x_train_gray.reshape((x_train_gray.shape[0], -1))
x_test_flat = x_test_gray.reshape((x_test_gray.shape[0], -1))

In [29]:
print(x_train_flat.shape,y_train.shape)

(50000, 1024) (50000, 1)


In [30]:
# Function to extract features from images using SIFT
def extract_features(images):
    sift = cv2.SIFT_create()
    descriptors = []
    for img in tqdm(images, desc="Extracting features"):
        kp, des = sift.detectAndCompute(img, None)
        if des is not None:
            descriptors.extend(des)
    return np.array(descriptors)

In [31]:
# Extract features from training images
train_features = extract_features(x_train_gray)

Extracting features: 100%|██████████| 50000/50000 [01:16<00:00, 656.80it/s]


In [32]:
# Perform K-means clustering
kmeans = KMeans(n_clusters=23) # Adjust number of clusters as needed
kmeans.fit(train_features)
visual_words = kmeans.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [33]:
# Function to represent images using Bag-of-Visual-Words
def image_representation(images, kmeans, visual_words):
    representations = []
    sift = cv2.SIFT_create()
    for img in tqdm(images, desc="Creating image representations"):
        _, des = sift.detectAndCompute(img, None)
        if des is not None:
            hist = np.zeros(len(visual_words))
            labels = kmeans.predict(des)
            for label in labels:
                hist[label] += 1
            representations.append(hist)
    return np.array(representations)

# Construct image representations for training and test images
X_train = image_representation(x_train_gray, kmeans, visual_words)
X_test = image_representation(x_test_gray, kmeans, visual_words)

Creating image representations: 100%|██████████| 10000/10000 [00:18<00:00, 547.65it/s]


In [34]:
X_train.shape,X_test.shape

((49905, 23), (9986, 23))

In [35]:
# Trim y_train to match the number of samples in X_train
y_train = y_train[:X_train.shape[0]]

# Ensure the number of samples in X_train matches the number of labels
assert X_train.shape[0] == y_train.shape[0], "Number of samples in features and labels do not match for training data."

# Trim y_test to match the number of samples in X_test
y_test = y_test[:X_test.shape[0]]

# Ensure the number of samples in X_test matches the number of labels
assert X_test.shape[0] == y_test.shape[0], "Number of samples in features and labels do not match for test data."


In [36]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((49905, 23), (9986, 23), (49905, 1), (9986, 1))

In [37]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [38]:
# Train Logistic Regression classifier
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [39]:
# Evaluate Logistic Regression classifier
y_pred_logistic_regression = logistic_regression.predict(X_test)
accuracy_logistic_regression = accuracy_score(y_test, y_pred_logistic_regression)
print("Logistic Regression Accuracy:", accuracy_logistic_regression)

Logistic Regression Accuracy: 0.106048467854997
